In [1]:
%load_ext autoreload
%autoreload 2

In [6]:
from absbox import API
localAPI = API("http://localhost:8081")

In [12]:
from absbox.local.generic import Generic

test01 = Generic(
    "Multiple Waterfall"
    ,{"cutoff":"2021-03-01","closing":"2021-06-15","firstPay":"2021-07-26"
     ,"payFreq":["DayOfMonth",20],"poolFreq":"MonthEnd","stated":"2030-01-01"}
    ,{'assets':[["Mortgage"
        ,{"originBalance":2200,"originRate":["fix",0.045],"originTerm":30
          ,"freq":"Monthly","type":"Level","originDate":"2021-02-01"}
          ,{"currentBalance":2200
          ,"currentRate":0.08
          ,"remainTerm":20
          ,"status":"current"}]]}
    ,(("acc01",{"balance":0}),)
    ,(("A1",{"balance":500
             ,"rate":0.07
             ,"originBalance":500
             ,"originRate":0.07
             ,"startDate":"2020-01-03"
             ,"rateType":{"Fixed":0.08}
             ,"bondType":{"Sequential":None}})
      ,("A2",{"balance":500
             ,"rate":0.07
             ,"originBalance":500
             ,"originRate":0.07
             ,"startDate":"2020-01-03"
             ,"rateType":{"Fixed":0.08}
             ,"bondType":{"Sequential":None}})      
      ,("B",{"balance":1000
             ,"rate":0.0
             ,"originBalance":1000
             ,"originRate":0.07
             ,"startDate":"2020-01-03"
             ,"rateType":{"Fixed":0.00}
             ,"bondType":{"Equity":None}
             }))
    ,(("trusteeFee",{"type":{"fixFee":30}}),)
    ,{"amortizing":[
         ["payFee",["acc01"],['trusteeFee']]
         ,["payInt","acc01",["A1","A2"]]
         ,["payPrin","acc01",["A1","A2"]]
         ,["payPrin","acc01",["B"]]
         ,["payResidual","acc01","B"]]
      ,"cleanUp":[]
      ,"endOfCollection":[]       # execute when collect money from pool
      ,("amortizing","defaulted"):[]   #execute when deal is `defaulted`
      ,("amortizing","accelerated"):[ #execute when deal is `accelerated`
         ["payFee",["acc01"],['trusteeFee']]
         ,["payInt","acc01",["A1","A2"]]
         ,["payPrin","acc01",["A1"]] 
         ,["payPrin","acc01",["A2"]]
         ,["payPrin","acc01",["B"]]
         ,["payResidual","acc01","B"]
      ] 
      }
    ,[["CollectedInterest","acc01"]
      ,["CollectedPrincipal","acc01"]
      ,["CollectedPrepayment","acc01"]
      ,["CollectedRecoveries","acc01"]]
    ,None
    ,None
    ,None
    ,{"AfterCollect":[
        {"condition":[("cumPoolDefaultedRate",),">",0.05],
        "effects":("newStatus","Accelerated"),
        "status":False,
        "curable":False} ]
      }
)

In [48]:
r['result']['status']

,Date,From,To
0,2022-01-31,Amortizing,Accelerated


In [51]:
r = localAPI.run(test01
               ,assumptions=[{"Inspect":[("MonthEnd",("poolBalance",))
                                          ,("MonthFirst",("bondBalance",))
                                          ,("MonthEnd",("trigger","AfterCollect", 0))]
                             }
                            ,{"CDR":0.5}]
               ,read=True)
r['result']['inspect']['<TriggersStatusAt:EndCollectionWF,0>']

,"<TriggersStatusAt:EndCollectionWF,0>"
Date,
2021-03-01,False
2021-03-31,False
2021-04-30,False
2021-05-31,False
2021-06-30,False
2021-07-31,False
2021-08-31,False
2021-09-30,False
2021-10-31,False


In [57]:
r['result']['inspect']['<CurrentBondBalance>']

,<CurrentBondBalance>
Date,
2021-03-01,2000
2021-04-01,2000
2021-05-01,2000
2021-06-01,2000
2021-07-01,2000
2021-08-01,2000
2021-09-01,2000
2021-10-01,2000
2021-11-01,2000
